# Problem statement 01

- Write a linear equation solver that will take in matrices $A$ and $b$ as inputs, and return the vector $x$ that solves the equation $Ax=b$.  Your function should catch errors in the inputs and return suitable error messages for different possible problems.
  - Time your solver to solve a random $10\times 10$ system of equations.  Compare the time taken against the `numpy.linalg.solve` function for the same inputs.

In [1]:
import numpy as np

### My linear equation solver function.

*Note: The following function works only for **square matrix A***

In [ ]:
def MySolver(A, b, w = 0):

    (m, m) = np.shape(A)
    if w != 0:
        aug = np.zeros((m+1, m), dtype=complex)
    else:
        aug = np.zeros((m+1, m))
    A = np.transpose(A)
    for j in range(m):
        aug[j] += A[j]
    aug[m] += b
    aug = np.transpose(aug)
# Formed an augmented matrix.
    for a in range(m):
        if aug[a][a] == 0:
            for i in range(a, m):
                if aug[i][a] != 0:
                    aug[[a, i]] = aug[[i, a]]
    for i in range(m):
        if aug[i][i] == 0:
            print('No solution')
            return
# Reformed augmented matrix to make sure of 
    def rref(X, k):
        (n,_) = np.shape(X)
        X[k] /= X[k][k]
        for i in range(k+1, n):
            for a in range(m):
                if X[a][a] == 0:
                    for w in range(a, m):
                        if X[w][a] != 0:
                            X[[a, w]] = X[[w, a]]
            X[i] -= X[i][k]*X[k]
        if k < n-1:
            rref(X, k+1)
# Formed the lower triangular matrix of the augmented matrix.
        else:
            for i in range(m-1, 0, -1):
                for j in range(1, i+1):
                    X[i-j] -= X[i]*X[i-j][i]
# Formed an identity matrix in place of 'A' in the augmented matrix.
        ans = np.transpose(X)[-1]
        for x in ans:
            x = str(x)
            if x == 'nan' or x == 'inf' or x == '-inf':
                return
            else:
                return list(ans)
# Got the last column matrix, which is the final answer.
    if (rref(aug, 0) == 'nan'):
        print('No solution.')
    else:
        return rref(aug, 0)


**Explanation:**

- This function can solve a set of linear equations, given that #equations = #variables.
- This is based on Gaussian method of solving linear equations.
- It will form an augmented matrix using the 2 given matrices.
- Then using row operations, the augmented matrix is reduced to **rref**(row reduced echlon form).
- Thus formed matrix will be an augmented matrix of identity matrix and the solution for the set of equations.
- It can also say if the given set of equations has solutions.
- It can also solve if the coefficients are complex numbers.

In [ ]:
y = 10
A = np.random.rand(y,y)
b = np.random.rand(y)

print(f'Shape of A = {np.shape(A)}')
print(f'Shape of b = {np.shape(b)}')

The above code is to create a 10 x 10 matrix with random entries.

In [ ]:
print(f'The solution of given matix is {MySolver(A,b)}')
%timeit MySolver(A,b)

- When I run the function with a 10X10 matrix as input, the time calculated by timeit function came around 690 micro seconds, which I feel is quite good.

In [ ]:
def NumpySolver(A, b):
    return np.linalg.solve(A, b)
print(NumpySolver(A, b))
%timeit NumpySolver(A, b)

**Explanation:**

- There's a module called **linalg** in numpy library, which has **solve** function that can solve linear equations. Using `np.linalg.solve()` I called that function.
- It took time of around 15 micro seconds for the same input as given to MySolver, which is 100 times more efficient than MySolver.

### Intelligent user input.

In [ ]:
def SolveInputdata():
    try:
        y = int(input('Enter the no. of equations to be solved: '))
        x = int(input('Enter no. of unknown variables: '))
    except:
        print('Enter integer only.')
    if x != y:
        print('Expected to have same no. of equations as that of no. of variables.')
    else:
        A = np.zeros((x, x))
        print("Enter the entries of matrix A row wise.")
        for i in range(x):
            for j in range(x):
                status = 'wrong'
                while status == 'wrong':
                    try:
                        status = 'correct'
                        A[i][j] = float(input(f'A[{i}][{j}] = '))
                    except:
                        status = 'wrong'
                        print("Enter the correct element.")
        print("Enter the entries of matrix B.")
        b = np.zeros(x)
        for i in range(x):
            status = 'wrong'
            while status == 'wrong':
                try:
                    status = 'correct'
                    b[i] = float(input(f'b[{i}] = '))
                except:
                    status = 'wrong'
                    print("Enter the correct element.")
                    b[i] = input(f'b[{i}] = ')
        
        print(f'A = {A}')
        print(f'b = {b}')
        return MySolver(A, b)

In [ ]:
SolveInputdata()

**Explanation:**

- This function takes user input, forms matrices, calles **MySolver** function with correct arguments and returns it, which is the solution.
- The function talks to user if there's any problem in input data.
- If A is not a square matrix, function ends.
- Also if entries of matrices are not the integers, it keeps taking that entry untill the user puts in the valid entry.

## Problem statement 02

- Given a circuit netlist in the form described above, read it in from a file, construct the appropriate matrices, and use the solver you have written above to obtain the voltages and currents in the circuit.  If you find AC circuits hard to handle, first do this for pure DC circuits, but you should be able to handle both voltage and current sources.

### Creating a .netlist file.

In [ ]:
with open('dcCkt.netlist', 'w') as f:
    f.write('''.circuit
V1 GND 1 dc 30
R1 1 2 4
R2 2 GND 6
R3 2 3 6
R4 3 GND 12
I1 3 4 dc 1
R5 4 GND 12
.end''')

**Explanation:**

- Using the basic write to file synatx, I created a file named 'dcCkt.netlist'.

### What is in the .netlist file.

In [ ]:
def datainfile(filename):
    with open(filename) as f:
        line = 'start'
        r = v = i = c = l = ac =0
        while line != '':
            line = f.readline()
            spl = line.split()
            try:
                if line[0] == 'V':
                    v += 1
                    if spl[3] == 'ac':
                        ac += 1
                        print(f'{spl[3]} voltage source(V{v}) of {float(spl[4])}V and phase {spl[5]} connected between nodes {spl[1]} and {spl[2]}')
                    else:
                        print(f'{spl[3]} voltage source(V{v}) of {float(spl[4])}V connected between nodes {spl[1]} and {spl[2]}')
                elif line[0] == 'R':
                    r += 1
                    print(f'Resistor(R{r}) of {float(spl[3])} ohms connected between nodes {spl[1]} and {spl[2]}')
                elif line[0] == 'C':
                    c += 1
                    print(f'Capacitor(C{c}) of {float(spl[3])}F connected between nodes {spl[1]} and {spl[2]}')
                elif line[0] == 'L':
                    l += 1
                    print(f'Inductor(L{l}) of {float(spl[3])}H connected between nodes {spl[1]} and {spl[2]}')
                elif line[0] == 'I':
                    i += 1
                    if spl[3] == 'ac':
                        ac += 1
                        print(f'{spl[3]} current source(I{i}) of {float(spl[4])}A and phase {spl[5]} connected between nodes {spl[1]} and {spl[2]}')
                    else:
                        print(f'{spl[3]} current source(I{i}) of {float(spl[4])}A connected between nodes {spl[1]} and {spl[2]}')
                for i in range(ac):
                    if line[4] == 'V':
                        print(f'Frequency of AC {spl[1]} = {spl[2]}')
                    if line[4] == 'I':
                        print(f'Frequency of {spl[1]} = {spl[2]}') 
            except:
                pass

In [ ]:
datainfile('acCkt2.netlist')

**Explanation:**

This function reads the content of the file and gives out all the data about the defined circuit in human readable format.

### Circuit solver using MNA

In [ ]:
import cmath

def mna(filename):
    m = 0
    with open(filename) as f:
        x = len(f.read().splitlines())
    # Getting dimensions of matrix.
    with open(filename) as f:
        maxi = 0
        for i in range(x):
            if f.readline().split() == ['.circuit']:
                line = ''
                while line.split() != ['.end']:
                    line = f.readline()
                    spl = line.split()
                    if line[0] == 'V':
                        m += 1
                    elif line[0] == 'R' or line[0] == 'C' or line[0] == 'L':
                        if spl[1] != 'GND' and spl[2] != 'GND':
                            maxi = max(int(spl[1][-1]), int(spl[2][-1]), maxi)
                        elif spl[1] == 'GND' or spl[2] != 'GND':
                            maxi = max(int(spl[2][-1]), maxi)
                        elif spl[1] != 'GND' or spl[2] == 'GND':
                            maxi = max(int(spl[1][-1]), maxi)
    # reading lines after .end line in file for ac circuits to get frequency.
                w = 0
                line = f.readline()
                if line != '' and line.split()[0] == '.ac':
                    w = int(line.split()[2])
    ac = 0
    m = maxi + m # matrix dimension.
    if w == 0:
        G = np.zeros((m,m))
    else:
        G = np.zeros((m, m), dtype=complex)
    if w == 0:
        I = np.zeros(m)
    else:
        I = np.zeros(m, dtype=complex)
    # matrix formation.
    with open(filename) as f:
        for i in range(x):
            if f.readline().split() == ['.circuit']:
                line = ''
                while line.split() != ['.end']:
                    line = f.readline()
                    spl = line.split()
    # Resistor data into matrix.
                    if line[0] == 'R' and spl[1] != 'GND' and spl[2] != 'GND':
                        G[int(spl[1][-1])][int(spl[1][-1])] += 1/float(spl[3])
                        G[int(spl[2][-1])][int(spl[2][-1])] += 1/float(spl[3])
                        G[int(spl[1][-1])][int(spl[2][-1])] += -1/float(spl[3])
                        G[int(spl[2][-1])][int(spl[1][-1])] += -1/float(spl[3])
                    elif line[0] == 'R' and spl[1] == 'GND':
                        G[int(spl[2][-1])][int(spl[2][-1])] += 1/float(spl[3])
                    elif line[0] == 'R' and spl[2] == 'GND':
                        G[int(spl[1][-1])][int(spl[1][-1])] += 1/float(spl[3])
                    
    # inductor data into matrix.
                    if line[0] == 'L' and spl[1] != 'GND' and spl[2] != 'GND':
                        G[int(spl[1][-1])][int(spl[1][-1])] += 1/(float(spl[3])*1j*w)
                        G[int(spl[2][-1])][int(spl[2][-1])] += 1/(float(spl[3])*1j*w)
                        G[int(spl[1][-1])][int(spl[2][-1])] += -1/(float(spl[3])*1j*w)
                        G[int(spl[2][-1])][int(spl[1][-1])] += -1/(float(spl[3])*1j*w)
                    elif line[0] == 'L' and spl[1] == 'GND':
                        G[int(spl[2][-1])][int(spl[2][-1])] += 1/(float(spl[3])*1j*w)
                    elif line[0] == 'L' and spl[2] == 'GND':
                        G[int(spl[1][-1])][int(spl[1][-1])] += 1/(float(spl[3])*1j*w)

    # capacitor data into matrix.
                    if line[0] == 'C' and spl[1] != 'GND' and spl[2] != 'GND':
                        G[int(spl[1][-1])][int(spl[1][-1])] += float(spl[3])*1j*w
                        G[int(spl[2][-1])][int(spl[2][-1])] += float(spl[3])*1j*w
                        G[int(spl[1][-1])][int(spl[2][-1])] += float(spl[3])*1j*w
                        G[int(spl[2][-1])][int(spl[1][-1])] += float(spl[3])*1j*w
                    elif line[0] == 'C' and spl[1] == 'GND':
                        G[int(spl[2][-1])][int(spl[2][-1])] += 1j*float(spl[3])*w
                    elif line[0] == 'C' and spl[2] == 'GND':
                        G[int(spl[1][-1])][int(spl[1][-1])] += 1j*float(spl[3])*w

    # voltage data into matrix(Auxilary currents).
                    elif line[0] == 'V' and spl[1] != 'GND' and spl[2] != 'GND':
                        for i in range(m):
                            if (G[i] == np.zeros(m)).all():
                                break
                        G[i][int(spl[1][-1])] += 1
                        G[int(spl[1][-1])][i] += 1
                        G[i][int(spl[2][-1])] += 1
                        G[int(spl[2][-1])][i] += -1
                        if spl[3] == 'dc':
                            I[i] += float(spl[4])
                        elif spl[3] == 'ac':
                            ac = 1
                            I[i] += cmath.rect(float(spl[4]), np.deg2rad(float(spl[5])))
                        print(f'{I[i]} is auxilary current through voltage source.')
                    elif line[0] == 'V' and spl[1] == 'GND':
                        for i in range(m):
                            if (G[i] == np.zeros(m, dtype = complex)).all():
                                break
                        G[i][int(spl[2][-1])] += 1
                        G[int(spl[2][-1])][i] += -1
                        if spl[3] == 'dc':
                            I[i] += float(spl[4])
                        elif spl[3] == 'ac':
                            ac = 1
                            I[i] += cmath.rect(float(spl[4]), np.deg2rad(float(spl[5])))
                        print(f'{i}th element in result is auxilary current through voltage source.')
                    elif line[0] == 'V' and spl[2] == 'GND':
                        for i in range(m):
                            if (G[i] == np.zeros(m, dtype = complex)).all():
                                break
                        G[int(spl[1][-1])] += 1
                        G[int(spl[1][-1])][i] += 1
                        if spl[3] == 'dc':
                            I[i] += float(spl[4])
                        elif spl[3] == 'ac':
                            ac = 1
                            I[i] += cmath.rect(float(spl[4]), np.deg2rad(float(spl[5])))
                        print(f'{I[i]} is auxilary current through voltage source.')

# current data into matrix.
                    elif line[0] == 'I' and spl[1] == 'GND':
                        if spl[3] == 'dc':
                            I[int(spl[2][-1])] += float(spl[4])
                        elif spl[3] == 'ac':
                            ac = 1
                            I[int(spl[2][-1])] += cmath.rect(float(spl[4]), np.deg2rad(float(spl[5])))
                    elif line[0] == 'I' and spl[2] == 'GND':
                        if spl[3] == 'dc':
                            I[int(spl[1][-1])] += -float(spl[4])
                        elif spl[3] == 'ac':
                            ac = 1
                            I[int(spl[1][-1])] += -cmath.rect(float(spl[4]), np.deg2rad(float(spl[5])))
                    elif line[0] == 'I' and spl[1] != 'GND' and spl[2] != 'GND':
                        if spl[3] == 'dc':
                            I[int(spl[1][-1])] += -float(spl[4])
                            I[int(spl[2][-1])] += float(spl[4])
                        elif spl[3] == 'ac':
                            ac = 1
                            I[int(spl[1][-1])] += -cmath.rect(float(spl[4]), np.deg2rad(float(spl[5])))
                            I[int(spl[2][-1])] += cmath.rect(float(spl[4]), np.deg2rad(float(spl[5])))
        print('And the remaining are the node voltages in the below list.')
    if ac == 0:
        return MySolver(G, I, w)
    else:
        try:
            return [cmath.polar(x) for x in MySolver(G, I, w)]
        except:
            pass

**Explanation:**

- Used cmath library to convert complex to phasor and vice versa.
- This function takes file name as input and gives out all the node voltages and auxilary currents in the circuit.
- I followed MNA(**Modified nodal analysis**) to build this function.
- I reads the values from the file and creates G and I matrices, solves them to give V matrix.
- I solves both dc and ac(with single frequency) circuits.

### Solution for a DC circuit.

In [ ]:
print(mna('dcCkt.netlist'))

**Explanation:**

The equations of the given circuit using nodal analysis are as follows:

$$
\begin{aligned}
V1 & - & 0 & = & 30\\
\frac{V2-V1}{R1} & + & \frac{V2-0}{R2} & + & \frac{V2-V3}{R3} & = & 0 \\
\frac{V3-V2}{R3} & + & \frac{V3-0}{R4} & + & 1 & = & 0 \\
\frac{V4-0}{R5} & - & 1 & = & 0
\end{aligned}
$$

The above 4 system of linear equations can be written in matrix form as follows:

$$
\begin{bmatrix}
1 & 0 & 0 & 0 \\
\frac{-1}{R1} & \frac{1}{R1}+\frac{1}{R2}+\frac{1}{R3} & \frac{-1}{R3}  & 0 \\
0 & \frac{-1}{R3} & \frac{1}{R3}+\frac{1}{R4} & 0 \\
0 & 0 & 0 & \frac{1}{R5}
\end{bmatrix}
\begin{bmatrix}
V1 \\
V2 \\
V3 \\
V4
\end{bmatrix}
=
\begin{bmatrix}
30 \\
0 \\
-1 \\
1
\end{bmatrix}
$$
This matrix is directly formed using MNA in the function.

### Solution for an AC circuit.

In [ ]:
mna('dcCkt.netlist')

**Explanation**

- The output values are in phasor form